# **Train Model**

In [6]:
# 1. فك أي درايف متوصّل
!fusermount -u /content/drive 2>/dev/null

# 2. امسح المجلد اللي عامل مشاكل
!rm -rf /content/drive

# 3. اعمل mount من جديد
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
!python --version
!pip -q install scikit-learn==1.3.2 pandas numpy matplotlib joblib


Python 3.12.11
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 104.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 88.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
imbalanced-learn 0.14.0 requires scikit-learn<2,>=1.4.2, but you have scikit-learn 1.3.2 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit

In [1]:
%pip -q install -U scikit-learn==1.6.1 xgboost==2.1.2 lightgbm==4.5.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 5.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.1 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.


In [2]:
import numpy, pandas, sklearn, scipy
print("NumPy:", numpy.__version__)
print("pandas:", pandas.__version__)
print("sklearn:", sklearn.__version__)
print("scipy:", scipy.__version__)


NumPy: 2.1.3
pandas: 2.2.3
sklearn: 1.6.1
scipy: 1.13.1


In [4]:
%pip install -U scipy==1.14.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 25.8 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1


In [8]:
import pandas as pd
WIDE_PATH = "/content/drive/MyDrive/Bloom Watch/processed/ANALYSIS_ready_WIDE_final.csv"
df = pd.read_csv(WIDE_PATH, nrows=3)
print("Columns sample:", list(df.columns)[:80])


Columns sample: ['region', 'Year', 'week', 'date', 'bloom_stage', 'month', 'doy', 'AirTemp_avg_final', 'AirTemp_max_final', 'AirTemp_min_final', 'Precipitation_final', 'Humidity_rel_final', 'VaporPressureDeficit_final', 'Evapotranspiration_ref_final', 'SpecificHumidity_final', 'SolarRadiation_sw_final', 'SurfacePressure_final']


In [4]:
# ============================
# ✅ Normalize with Superbloom as a distinct label + include region
# ============================
import re, os, json, joblib, warnings
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GroupKFold, cross_validate
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC
from sklearn.dummy import DummyClassifier

# ---- المسارات
WIDE_PATH = "/content/drive/MyDrive/Bloom Watch/processed/ANALYSIS_ready_WIDE_final.csv"
ARTIFACT_DIR = "/content/drive/MyDrive/Bloom Watch/processed/models_bloom_hier"
os.makedirs(ARTIFACT_DIR, exist_ok=True)
BIN_MODEL_PATH = os.path.join(ARTIFACT_DIR, "gate_bloom_binary.joblib")
STG_MODEL_PATH = os.path.join(ARTIFACT_DIR, "stage_multiclass.joblib")
META_PATH      = os.path.join(ARTIFACT_DIR, "meta.json")

# ---- أعمدة
DATE_COL   = "date"
YEAR_COL   = "Year"
MONTH_COL  = "month"
WEEK_COL   = "week"
TARGET_STAGE_COL = "bloom_stage"

# ندخل region كـ categorical feature
CATEGORICAL_FEATURES = ["region"]

# توحيد أعمدة المناخ (نفس الـ map القديم بيغطي *_final)
CANDIDATE_MAP = {
    "AirTemp_avg": ["AirTemp_avg_final","AirTemp_avg","T2M_final","T2M"],
    "AirTemp_max": ["AirTemp_max_final","AirTemp_max","T2M_MAX_final","T2M_MAX"],
    "AirTemp_min": ["AirTemp_min_final","AirTemp_min","T2M_MIN_final","T2M_MIN"],
    "Precipitation": ["Precipitation_final","Precipitation","PRECTOTCORR_final","PRECTOTCORR"],
    "Humidity_rel": ["Humidity_rel_final","Humidity_rel","RH2M_final","RH2M"],
    "VaporPressureDeficit": ["VaporPressureDeficit_final","VaporPressureDeficit","VPD_final","VPD"],
    "Evapotranspiration_ref": ["Evapotranspiration_ref_final","Evapotranspiration_ref","ET0_final","ET0"],
    "SpecificHumidity": ["SpecificHumidity_final","SpecificHumidity","QV2M_final","QV2M"],
    "SolarRadiation_sw": ["SolarRadiation_sw_final","SolarRadiation_sw","ALLSKY_SFC_SW_DWN_final","ALLSKY_SFC_SW_DWN"],
    "SurfacePressure": ["SurfacePressure_final","SurfacePressure","PS_final","PS"],
}
TIME_CANDIDATE_MAP = {
    "month": ["month"],
    "Year": ["Year","year"],
    "doy": ["doy","day_of_year"],
    "week_of_year": ["week_of_year","isoweek","week"],
    "week": ["week","week_iso","week_of_year"]
}

NUMERIC_FEATURES = [
    "AirTemp_avg", "AirTemp_max", "AirTemp_min",
    "Precipitation", "Humidity_rel", "VaporPressureDeficit",
    "Evapotranspiration_ref", "SpecificHumidity",
    "SolarRadiation_sw", "SurfacePressure"
]
NUMERIC_EXTRA_TIME = ["week_of_year", "doy", "month"]

def ensure_time_fields(df: pd.DataFrame) -> pd.DataFrame:
    if DATE_COL in df.columns:
        df[DATE_COL] = pd.to_datetime(df[DATE_COL])
        if "month" not in df.columns: df["month"] = df[DATE_COL].dt.month
        if "Year"  not in df.columns: df["Year"]  = df[DATE_COL].dt.year
        df["doy"] = df[DATE_COL].dt.dayofyear
        df["week_of_year"] = df[DATE_COL].dt.isocalendar().week.astype(int)
        if "week" not in df.columns: df["week"] = df["week_of_year"]
    else:
        for k, candidates in TIME_CANDIDATE_MAP.items():
            if k not in df.columns:
                for c in candidates:
                    if c in df.columns:
                        df[k] = df[c]; break
        need = ["month","Year","doy","week_of_year","week"]
        missing = [c for c in need if c not in df.columns]
        if missing:
            raise ValueError(f"Missing time columns: {missing} (or provide '{DATE_COL}')")
    return df

def unify_feature_names(df: pd.DataFrame) -> pd.DataFrame:
    dfu = df.copy()
    for std_name, candidates in CANDIDATE_MAP.items():
        found = None
        for c in candidates:
            if c in dfu.columns:
                found = c; break
        if found is None:
            raise KeyError(f"Missing feature for '{std_name}'. Print columns to inspect.")
        if found != std_name:
            dfu = dfu.rename(columns={found: std_name})
    dfu = ensure_time_fields(dfu)
    return dfu

# ---------- تطبيع أسماء المراحل مع Superbloom منفصلة + دعم LMoB ----------
# أمثلة محتملة: "SoB | Super", "SoB / Superbloom", "Start of Superbloom", "Superbloom Start", ...
def normalize_stage_label(s: str) -> str:
    if not isinstance(s, str) or s.strip() == "":
        return s
    t = s.strip().lower()

    # نظّف فواصل
    t = re.sub(r"\s*\|\s*", "|", t)
    t = re.sub(r"\s*/\s*", "/", t)

    # كلمات مفتاحية
    is_super = ("superbloom" in t) or (re.search(r"\bsuper\b", t) is not None)

    # خرائط مباشرة شائعة
    direct = {
        "sob": "SoB", "start bloom": "SoB", "start of bloom": "SoB",
        "mob": "MoB", "mid bloom": "MoB", "mid of bloom": "MoB",
        "lmob": "LMoB", "late mid": "LMoB", "late mid of bloom": "LMoB",
        "pob": "PoB", "peak bloom": "PoB", "peak of bloom": "PoB",
        "lpob": "LPoB", "late peak": "LPoB", "late peak of bloom": "LPoB",
        "eob": "EoB", "end of bloom (eob)": "EoB", "end of bloom": "EoB",
        "no bloom": "NoBloom", "nobloom": "NoBloom"
    }
    if t in direct:
        lbl = direct[t]
        # لو SoB ومعاه super → عايزين SoB_super
        if lbl == "SoB" and is_super:
            return "SoB_super"
        return lbl

    # صيغ مركبة
    if "start" in t and "bloom" in t:
        return "SoB_super" if is_super else "SoB"
    if "mid" in t and "late" in t:
        return "LMoB"
    if ("mid" in t) or ("mob" in t):
        return "MoB"
    if ("peak" in t) or ("pob" in t):
        return "LPoB" if "late" in t else "PoB"
    if "end" in t or "eob" in t:
        return "EoB"
    if "no bloom" in t or "nobloom" in t:
        return "NoBloom"

    # أنماط زي: "sob / superbloom" أو "sob | super"
    if "sob" in t or "start" in t:
        return "SoB_super" if is_super else "SoB"

    # لو مش متعرف → ارجعي النص الأصلي لكن بحروف قياسية أولية
    return s.strip()

def make_binary_label_from_stage(df: pd.DataFrame, col: str) -> pd.Series:
    s = df[col].astype(str).apply(normalize_stage_label)
    return np.where(s == "NoBloom", "NoBloom", "Bloom")

def _prepare_stage_df(df: pd.DataFrame) -> pd.DataFrame:
    df_st = df[df["month"].between(2,6)].copy()
    df_st[TARGET_STAGE_COL] = df_st[TARGET_STAGE_COL].apply(normalize_stage_label)
    # استبعد NoBloom من مهمة الـStage
    df_st = df_st[df_st[TARGET_STAGE_COL] != "NoBloom"]
    df_st = df_st[~df_st[TARGET_STAGE_COL].isna()]
    return df_st

def build_preprocessor():
    return ColumnTransformer(
        transformers=[
            ("num", StandardScaler(), NUMERIC_FEATURES + NUMERIC_EXTRA_TIME),
            ("cat", OneHotEncoder(handle_unknown="ignore"), CATEGORICAL_FEATURES),
        ],
        remainder="drop"
    )

def evaluate_and_select(task_name, X, y, groups, scoring_pref="f1_macro"):
    pre = build_preprocessor()
    gkf = GroupKFold(n_splits=5)
    scoring = {"acc":"accuracy", "bal_acc":"balanced_accuracy", "f1_macro":"f1_macro"}

    # موديلات: (نِسِيب XGB/LGBM للـStage دلوقتي لتجنب مشاكل اللابلز)
    if task_name == "binary":
        zoo = {
            "RF":  RandomForestClassifier(n_estimators=800, min_samples_leaf=2, class_weight="balanced_subsample", n_jobs=-1, random_state=42),
            "HGB": HistGradientBoostingClassifier(max_leaf_nodes=31, learning_rate=0.08, min_samples_leaf=20),
            "ET":  ExtraTreesClassifier(n_estimators=800, min_samples_leaf=2, class_weight="balanced_subsample", n_jobs=-1, random_state=42),
            "CalibratedLR": CalibratedClassifierCV(LogisticRegression(max_iter=2000, class_weight="balanced"), method="sigmoid", cv=3),
            "CalibratedLinearSVC": CalibratedClassifierCV(LinearSVC(class_weight="balanced"), method="sigmoid", cv=3),
        }
    else:
        zoo = {
            "RF":  RandomForestClassifier(n_estimators=1200, min_samples_leaf=2, class_weight="balanced_subsample", n_jobs=-1, random_state=42),
            "HGB": HistGradientBoostingClassifier(max_leaf_nodes=63, learning_rate=0.06, min_samples_leaf=10),
            "ET":  ExtraTreesClassifier(n_estimators=1200, min_samples_leaf=2, class_weight="balanced_subsample", n_jobs=-1, random_state=42),
        }

    rows, best_pipe, best_score = [], None, -np.inf
    for name, clf in zoo.items():
        pipe = Pipeline([("pre", pre), ("clf", clf)])
        try:
            cv = cross_validate(pipe, X, y, groups=groups, scoring=scoring, cv=gkf, n_jobs=-1, return_estimator=True)
            row = {
                "model": name,
                "acc":      float(np.mean(cv["test_acc"])),
                "bal_acc":  float(np.mean(cv["test_bal_acc"])),
                "f1_macro": float(np.mean(cv["test_f1_macro"]))
            }
            rows.append(row)
            score_now = row[scoring_pref]
            if score_now > best_score:
                best_score = score_now
                best_idx = int(np.argmax(cv["test_f1_macro"]))
                best_pipe = cv["estimator"][best_idx]
        except Exception as e:
            warnings.warn(f"[{task_name}] {name} failed: {e}")
            continue

    lb = pd.DataFrame(rows).sort_values(by=scoring_pref, ascending=False)
    return best_pipe, lb

def train_with_model_selection(scoring_pref_bin="f1_macro", scoring_pref_stage="f1_macro"):
    raw = pd.read_csv(WIDE_PATH)
    df  = unify_feature_names(raw)

    # ===== Binary داخل 2..6 =====
    df_bin = df[df["month"].between(2,6)].copy()
    y_bin  = make_binary_label_from_stage(df_bin, TARGET_STAGE_COL)
    feats  = NUMERIC_FEATURES + NUMERIC_EXTRA_TIME + CATEGORICAL_FEATURES
    X_bin  = df_bin[feats].copy()
    g_bin  = df_bin["Year"]

    if (y_bin == "Bloom").all():
        print("⚠️ كل أمثلة التدريب داخل [2..6] = Bloom. سنعتمد على Gate Rule (2..6) + Dummy Bloom داخل النطاق.")
        pre = build_preprocessor()
        best_bin = Pipeline([("pre", pre), ("clf", DummyClassifier(strategy="constant", constant="Bloom"))]).fit(X_bin, y_bin)
        lb_bin = pd.DataFrame([{"model":"Dummy(Bloom)","acc":1.0,"bal_acc":0.5,"f1_macro":(2/3)}])
    else:
        print("\n🔎 Selecting best BINARY model ...")
        best_bin, lb_bin = evaluate_and_select("binary", X_bin, y_bin, g_bin, scoring_pref=scoring_pref_bin)

    joblib.dump(best_bin, BIN_MODEL_PATH)
    print("✅ Saved best binary model:", BIN_MODEL_PATH)
    display(lb_bin)

    # ===== Stage (تطبيع + Superbloom مستقل + استبعاد NoBloom) =====
    df_st = _prepare_stage_df(df)
    feats  = NUMERIC_FEATURES + NUMERIC_EXTRA_TIME + CATEGORICAL_FEATURES
    X_stg  = df_st[feats].copy()
    y_stg  = df_st[TARGET_STAGE_COL].copy()
    g_stg  = df_st["Year"]
    stage_labels = sorted(y_stg.unique().tolist())
    print("Stage labels after normalization:", stage_labels)

    print("\n🔎 Selecting best STAGE model ...")
    best_stg, lb_stg = evaluate_and_select("stage", X_stg, y_stg, g_stg, scoring_pref=scoring_pref_stage)
    joblib.dump(best_stg, STG_MODEL_PATH)
    print("✅ Saved best stage model:", STG_MODEL_PATH)
    display(lb_stg)

    meta = {
        "numeric_features": NUMERIC_FEATURES,
        "categorical_features": CATEGORICAL_FEATURES,
        "time_features": NUMERIC_EXTRA_TIME,
        "binary_labels": ["NoBloom","Bloom"],
        "stage_labels": stage_labels,
        "leaderboards": {"binary": lb_bin.to_dict(orient="records"),
                         "stage":  lb_stg.to_dict(orient="records")},
        "scoring_pref": {"binary": scoring_pref_bin, "stage": scoring_pref_stage}
    }
    with open(META_PATH, "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)
    print("✅ Saved meta:", META_PATH)

    return lb_bin, lb_stg

# 🏁 شغّلي التدريب بالاختيار التلقائي
lb_bin, lb_stg = train_with_model_selection()



🔎 Selecting best BINARY model ...
✅ Saved best binary model: /content/drive/MyDrive/Bloom Watch/processed/models_bloom_hier/gate_bloom_binary.joblib


,model,acc,bal_acc,f1_macro
2,ET,0.956428,0.953527,0.954813
0,RF,0.953409,0.949681,0.951697
1,HGB,0.948580,0.943335,0.946662
4,CalibratedLinearSVC,0.856858,0.849792,0.850246
3,CalibratedLR,0.810161,0.799063,0.799243


Stage labels after normalization: ['EoB', 'LPoB', 'MoB', 'PoB', 'SoB', 'SoB_super']

🔎 Selecting best STAGE model ...
✅ Saved best stage model: /content/drive/MyDrive/Bloom Watch/processed/models_bloom_hier/stage_multiclass.joblib


,model,acc,bal_acc,f1_macro
0,RF,0.705438,0.657396,0.646390
2,ET,0.693637,0.652647,0.641697
1,HGB,0.673522,0.617191,0.605106


✅ Saved meta: /content/drive/MyDrive/Bloom Watch/processed/models_bloom_hier/meta.json


In [5]:
import joblib

# تحميل الموديلات
binary_model = joblib.load("/content/drive/MyDrive/Bloom Watch/processed/models_bloom_hier/gate_bloom_binary.joblib")
stage_model  = joblib.load("/content/drive/MyDrive/Bloom Watch/processed/models_bloom_hier/stage_multiclass.joblib")

# تحميل الميتاداتا
import json
with open("/content/drive/MyDrive/Bloom Watch/processed/models_bloom_hier/meta.json","r") as f:
    meta = json.load(f)


## **Test**

In [9]:
# ============================
# 🧪 أدخلي قيمك يدوياً وجربي الموديل
# يتطلب أن تكون دالة predict_bloom مُعرفة بالفعل (من الخلية السابقة)
# ============================

import pandas as pd

def predict_one_case(
    date:str,
    region:str,
    AirTemp_avg:float,
    AirTemp_max:float,
    AirTemp_min:float,
    Precipitation:float,
    Humidity_rel:float,
    VaporPressureDeficit:float,
    Evapotranspiration_ref:float,
    SpecificHumidity:float,
    SolarRadiation_sw:float,
    SurfacePressure:float,
    prob_threshold:float=0.5
):
    """
    ادخلي قيم حالة واحدة، والدالة هترجع DataFrame في سطر واحد بالنتيجة.
    ملاحظة: بنستخدم أسماء الأعمدة بنسخة *_final عشان تتوافق مع CANDIDATE_MAP.
    """
    row = {
        "region": region,
        "date": date,  # YYYY-MM-DD
        "AirTemp_avg_final": AirTemp_avg,
        "AirTemp_max_final": AirTemp_max,
        "AirTemp_min_final": AirTemp_min,
        "Precipitation_final": Precipitation,
        "Humidity_rel_final": Humidity_rel,
        "VaporPressureDeficit_final": VaporPressureDeficit,
        "Evapotranspiration_ref_final": Evapotranspiration_ref,
        "SpecificHumidity_final": SpecificHumidity,
        "SolarRadiation_sw_final": SolarRadiation_sw,
        "SurfacePressure_final": SurfacePressure,
    }
    df = pd.DataFrame([row])
    pred = predict_bloom(df, prob_threshold=prob_threshold).copy()
    # عرض أنيق
    display(
        pred[[
            "region","date","bloom_flag","bloom_prob","bloom_stage","stage_prob"
        ]]
    )
    return pred

def predict_batch(cases:list, prob_threshold:float=0.5):
    """
    cases: قائمة قواميس، كل قاموس بنفس مفاتيح predict_one_case
    """
    df = pd.DataFrame(cases)
    pred = predict_bloom(df, prob_threshold=prob_threshold).copy()
    display(pred[["region","date","bloom_flag","bloom_prob","bloom_stage","stage_prob"]])
    return pred


In [10]:
_ = predict_one_case(
    date="2024-03-20",   # مارس داخل [2..6] → مؤهل لـ Bloom حسب القاعدة
    region="California",
    AirTemp_avg=16.5,
    AirTemp_max=23.0,
    AirTemp_min=10.2,
    Precipitation=0.8,          # mm/day
    Humidity_rel=58,            # %
    VaporPressureDeficit=1.1,   # kPa
    Evapotranspiration_ref=3.2, # mm/day
    SpecificHumidity=0.0062,    # kg/kg
    SolarRadiation_sw=230,      # W/m²
    SurfacePressure=101.2,      # kPa
    prob_threshold=0.5          # غيريها لو عايزة حساسية أعلى/أقل
)


,region,date,bloom_flag,bloom_prob,bloom_stage,stage_prob
0,California,2024-03-20,NoBloom,0.201388,-,0.0


# **Retrain Model**

In [13]:
# 👇 ضيفيها أعلى خلية التدريب v2 (مرة واحدة)
import numpy as np
import pandas as pd

def _safe_int_month(m):
    # يحوّل أي قيمة (tuple/list/ndarray/str/float) إلى int بأمان
    if isinstance(m, (tuple, list, np.ndarray)) and len(m) > 0:
        m = m[0]
    try:
        return int(m)
    except Exception:
        m2 = pd.to_numeric(m, errors="coerce")
        if pd.isna(m2):
            raise ValueError(f"Cannot convert month value {m!r} to int")
        return int(m2)


In [16]:
# ==========================================
# Retrain v2 — وقت/تاريخ أقوى + Priors + Z-scores + فصل رقمي/فئوي
# ==========================================
import os, json, joblib, warnings, numpy as np, pandas as pd, re
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GroupKFold, cross_validate
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC
from sklearn.dummy import DummyClassifier

# ---------- Paths ----------
WIDE_PATH   = "/content/drive/MyDrive/Bloom Watch/processed/ANALYSIS_ready_WIDE_final.csv"
ARTIFACT_DIR= "/content/drive/MyDrive/Bloom Watch/processed/models_bloom_hier"
os.makedirs(ARTIFACT_DIR, exist_ok=True)
BIN_MODEL_PATH = os.path.join(ARTIFACT_DIR, "gate_bloom_binary.joblib")
STG_MODEL_PATH = os.path.join(ARTIFACT_DIR, "stage_multiclass.joblib")
META_PATH      = os.path.join(ARTIFACT_DIR, "meta.json")

# ---------- Columns / Mapping ----------
DATE_COL, YEAR_COL, MONTH_COL, WEEK_COL = "date","Year","month","week"
TARGET_STAGE_COL = "bloom_stage"

# لو عندك فئات أخرى غير region أضيفيها هنا
CATEGORICAL_FEATURES = ["region"]

# توحيد أسماء المناخ (ندور على أول اسم موجود ونحوّله للاسم القياسي)
CANDIDATE_MAP = {
    "AirTemp_avg": ["AirTemp_avg_final","AirTemp_avg","T2M_final","T2M"],
    "AirTemp_max": ["AirTemp_max_final","AirTemp_max","T2M_MAX_final","T2M_MAX"],
    "AirTemp_min": ["AirTemp_min_final","AirTemp_min","T2M_MIN_final","T2M_MIN"],
    "Precipitation": ["Precipitation_final","Precipitation","PRECTOTCORR_final","PRECTOTCORR"],
    "Humidity_rel": ["Humidity_rel_final","Humidity_rel","RH2M_final","RH2M"],
    "VaporPressureDeficit": ["VaporPressureDeficit_final","VaporPressureDeficit","VPD_final","VPD"],
    "Evapotranspiration_ref": ["Evapotranspiration_ref_final","Evapotranspiration_ref","ET0_final","ET0"],
    "SpecificHumidity": ["SpecificHumidity_final","SpecificHumidity","QV2M_final","QV2M"],
    "SolarRadiation_sw": ["SolarRadiation_sw_final","SolarRadiation_sw","ALLSKY_SFC_SW_DWN_final","ALLSKY_SFC_SW_DWN"],
    "SurfacePressure": ["SurfacePressure_final","SurfacePressure","PS_final","PS"],
}
NUMERIC_BASE = list(CANDIDATE_MAP.keys())

# ---------- Helpers ----------
def ensure_time_fields(df: pd.DataFrame) -> pd.DataFrame:
    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"])
        if "month" not in df.columns: df["month"] = df["date"].dt.month
        if "Year"  not in df.columns: df["Year"]  = df["date"].dt.year
        if "doy"   not in df.columns: df["doy"]   = df["date"].dt.dayofyear
        if "week_of_year" not in df.columns:
            df["week_of_year"] = df["date"].dt.isocalendar().week.astype(int)
        if "week" not in df.columns: df["week"] = df["week_of_year"]
    else:
        need = ["month","Year","doy","week_of_year","week"]
        missing = [c for c in need if c not in df.columns]
        if missing: raise ValueError(f"Missing time cols: {missing} or provide 'date'.")
    # تأكيد month int
    df["month"] = pd.to_numeric(df["month"], errors="coerce").astype("Int64").astype(int)
    return df

def unify_feature_names(df: pd.DataFrame) -> pd.DataFrame:
    dfu = df.copy()
    for std, cand in CANDIDATE_MAP.items():
        found = None
        for c in cand:
            if c in dfu.columns:
                found = c; break
        if found is None:
            raise KeyError(f"Missing feature for '{std}'. Check columns.")
        if found != std: dfu = dfu.rename(columns={found: std})
    return ensure_time_fields(dfu)

def normalize_stage_label(s: str) -> str:
    if not isinstance(s,str) or s.strip()=="":
        return s
    t = s.lower().strip()
    t = re.sub(r"\s*\|\s*", "|", t); t = re.sub(r"\s*/\s*", "/", t)
    is_super = ("superbloom" in t) or (re.search(r"\bsuper\b", t) is not None)
    direct = {
        "sob":"SoB","start bloom":"SoB","start of bloom":"SoB",
        "mob":"MoB","mid bloom":"MoB",
        "lmob":"LMoB","late mid":"LMoB","late mid of bloom":"LMoB",
        "pob":"PoB","peak bloom":"PoB","peak of bloom":"PoB",
        "lpob":"LPoB","late peak":"LPoB","late peak of bloom":"LPoB",
        "eob":"EoB","end of bloom":"EoB","end of bloom (eob)":"EoB",
        "no bloom":"NoBloom","nobloom":"NoBloom"
    }
    if t in direct:
        lbl = direct[t];
        return "SoB_super" if (lbl=="SoB" and is_super) else lbl
    if "start" in t and "bloom" in t: return "SoB_super" if is_super else "SoB"
    if "late" in t and "mid" in t:    return "LMoB"
    if "mid" in t:                    return "MoB"
    if "late" in t and "peak" in t:   return "LPoB"
    if "peak" in t:                   return "PoB"
    if "end" in t or "eob" in t:      return "EoB"
    if "no bloom" in t or "nobloom" in t: return "NoBloom"
    if "sob" in t or "start" in t:    return "SoB_super" if is_super else "SoB"
    return s.strip()

def make_binary_from_stage(df, col=TARGET_STAGE_COL):
    s = df[col].astype(str).apply(normalize_stage_label)
    return np.where(s=="NoBloom","NoBloom","Bloom")

def add_time_encodings(df):
    df["m_sin"] = np.sin(2*np.pi*(df["month"]-1)/12)
    df["m_cos"] = np.cos(2*np.pi*(df["month"]-1)/12)
    df["w_sin"] = np.sin(2*np.pi*(df["week_of_year"]-1)/53)
    df["w_cos"] = np.cos(2*np.pi*(df["week_of_year"]-1)/53)
    df["d_sin"] = np.sin(2*np.pi*(df["doy"]-1)/366)
    df["d_cos"] = np.cos(2*np.pi*(df["doy"]-1)/366)
    return df

def _safe_int_month(m):
    if isinstance(m, (tuple, list, np.ndarray)) and len(m) > 0:
        m = m[0]
    try:
        return int(m)
    except Exception:
        m2 = pd.to_numeric(m, errors="coerce")
        if pd.isna(m2):
            raise ValueError(f"Cannot convert month value {m!r} to int")
        return int(m2)

def build_priors(full_df):
    df = full_df.copy()
    df["month"] = pd.to_numeric(df["month"], errors="coerce").astype("Int64").astype(int)
    df[TARGET_STAGE_COL] = df[TARGET_STAGE_COL].apply(normalize_stage_label)

    # Bloom prior by month
    df["_is_bloom"] = (df[TARGET_STAGE_COL] != "NoBloom")
    bp = df.groupby("month")["_is_bloom"].mean().rename_axis("month").reset_index()
    bloom_prior_by_month = {_safe_int_month(m): float(p) for m,p in zip(bp["month"], bp["_is_bloom"])}

    # Stage prior داخل الـ Bloom فقط
    st = df[df[TARGET_STAGE_COL] != "NoBloom"]
    stage_counts = st.groupby(["month", TARGET_STAGE_COL]).size()
    stage_probs = stage_counts.groupby(level=0).apply(lambda s: s/s.sum()).unstack(fill_value=0.0)

    stage_prior_by_month = {}
    for m in stage_probs.index:
        mi = _safe_int_month(m)
        stage_prior_by_month[mi] = {str(col): float(stage_probs.loc[m, col]) for col in stage_probs.columns}

    stages = sorted([str(c) for c in stage_probs.columns])
    return bloom_prior_by_month, stage_prior_by_month, stages

def add_priors_columns(df, bloom_prior_by_month, stage_prior_by_month, stages):
    df["prior_bloom_month"] = df["month"].map(bloom_prior_by_month).fillna(0.0)
    for stg in stages:
        df[f"prior_{stg}_month"] = df["month"].map({m:d.get(stg,0.0) for m,d in stage_prior_by_month.items()}).fillna(0.0)
    prior_cols = ["prior_bloom_month"] + [f"prior_{stg}_month" for stg in stages]
    return df, prior_cols

def add_monthly_anomalies(df, full_df, feat_list):
    z_cols = []
    stats = full_df.groupby("month")[feat_list].agg(['mean','std'])
    stats.columns = ['_'.join(c) for c in stats.columns]
    merged = df.merge(stats, left_on="month", right_index=True, how="left")
    for f in feat_list:
        zc = f"{f}_z"
        merged[zc] = (merged[f] - merged[f"{f}_mean"]) / merged[f"{f}_std"].replace(0,np.nan)
        z_cols.append(zc)
    drop_cols = [c for c in merged.columns if c.endswith("_mean") or c.endswith("_std")]
    return merged.drop(columns=drop_cols), z_cols

def hard_gate_month(m): return int(m) in [2,3,4,5,6]

# ---------- Model Zoo ----------
def get_binary_candidates():
    return {
        "ET":  ExtraTreesClassifier(n_estimators=900, min_samples_leaf=2, class_weight="balanced_subsample", n_jobs=-1, random_state=42),
        "RF":  RandomForestClassifier(n_estimators=900, min_samples_leaf=2, class_weight="balanced_subsample", n_jobs=-1, random_state=42),
        "HGB": HistGradientBoostingClassifier(max_leaf_nodes=63, learning_rate=0.06, min_samples_leaf=10),
        "CalibratedLinearSVC": CalibratedClassifierCV(LinearSVC(class_weight="balanced"), method="sigmoid", cv=3),
        "CalibratedLR": CalibratedClassifierCV(LogisticRegression(max_iter=2000, class_weight="balanced"), method="sigmoid", cv=3),
    }

def get_stage_candidates():
    return {
        "RF":  RandomForestClassifier(n_estimators=1400, min_samples_leaf=2, class_weight="balanced_subsample", n_jobs=-1, random_state=42),
        "ET":  ExtraTreesClassifier(n_estimators=1400, min_samples_leaf=2, class_weight="balanced_subsample", n_jobs=-1, random_state=42),
        "HGB": HistGradientBoostingClassifier(max_leaf_nodes=63, learning_rate=0.06, min_samples_leaf=10),
    }

def build_preprocessor(num_cols, cat_cols):
    return ColumnTransformer(
        transformers=[
            ("num", StandardScaler(), num_cols),
            ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ],
        remainder="drop"
    )

def evaluate_and_select(task, X, y, groups, num_cols, cat_cols, scoring_pref="f1_macro"):
    gkf = GroupKFold(n_splits=5)
    scoring = {"acc":"accuracy","bal_acc":"balanced_accuracy","f1_macro":"f1_macro"}
    zoo = get_binary_candidates() if task=="binary" else get_stage_candidates()
    rows, best, best_s = [], None, -np.inf
    for name, clf in zoo.items():
        pipe = Pipeline([("pre", build_preprocessor(num_cols, cat_cols)), ("clf", clf)])
        try:
            cv = cross_validate(pipe, X, y, groups=groups, scoring=scoring, cv=gkf, n_jobs=-1, return_estimator=True)
            row = {"model":name,
                   "acc":float(np.mean(cv["test_acc"])),
                   "bal_acc":float(np.mean(cv["test_bal_acc"])),
                   "f1_macro":float(np.mean(cv["test_f1_macro"]))}
            rows.append(row)
            s = row[scoring_pref]
            if s > best_s:
                best_s = s
                best = cv["estimator"][int(np.argmax(cv["test_f1_macro"]))]
        except Exception as e:
            warnings.warn(f"[{task}] {name} failed: {e}")
    lb = pd.DataFrame(rows)
    if not lb.empty:
        lb = lb.sort_values(by=scoring_pref, ascending=False)
    return best, lb

# ---------- Training ----------
def train_v2(scoring_pref_bin="f1_macro", scoring_pref_stage="f1_macro"):
    raw = pd.read_csv(WIDE_PATH)
    df  = unify_feature_names(raw)
    df[TARGET_STAGE_COL] = df[TARGET_STAGE_COL].apply(normalize_stage_label)

    # priors (من الداتا نفسها)
    bloom_prior_by_month, stage_prior_by_month, stage_set = build_priors(df)

    # time encodings
    df = add_time_encodings(df)

    # priors كأعمدة
    df, prior_cols = add_priors_columns(df, bloom_prior_by_month, stage_prior_by_month, stage_set)

    # anomalies شهريًا (Z-scores) للمناخ
    df, z_cols = add_monthly_anomalies(df, df, NUMERIC_BASE)

    # lists مفصولة: رقمي / فئوي
    time_cols = ["m_sin","m_cos","w_sin","w_cos","d_sin","d_cos"]
    feats_num = NUMERIC_BASE + z_cols + prior_cols + time_cols
    feats_cat = CATEGORICAL_FEATURES

    # ===== Binary داخل [2..6] =====
    df_bin = df[df["month"].between(2,6)].copy()
    y_bin  = make_binary_from_stage(df_bin)
    X_bin  = df_bin[feats_num + feats_cat].copy()
    g_bin  = df_bin["Year"]

    if (y_bin == "Bloom").all():
        print("⚠️ داخل [2..6] كل الأمثلة Bloom → Gate rule + Dummy.")
        pre = build_preprocessor(feats_num, feats_cat)
        best_bin = Pipeline([("pre", pre),
                             ("clf", DummyClassifier(strategy='constant', constant='Bloom'))]).fit(X_bin, y_bin)
        lb_bin = pd.DataFrame([{"model":"Dummy(Bloom)","acc":1.0,"bal_acc":0.5,"f1_macro":(2/3)}])
    else:
        print("🔎 Selecting best BINARY model (v2)...")
        best_bin, lb_bin = evaluate_and_select("binary", X_bin, y_bin, g_bin,
                                               num_cols=feats_num, cat_cols=feats_cat,
                                               scoring_pref=scoring_pref_bin)
    joblib.dump(best_bin, BIN_MODEL_PATH)
    print("✅ Saved best binary model →", BIN_MODEL_PATH)
    try:
        display(lb_bin if not lb_bin.empty else pd.DataFrame([{"note":"no models succeeded"}]))
    except Exception:
        print(lb_bin.head() if not lb_bin.empty else "no models succeeded")

    # ===== Stage داخل [2..6] (بدون NoBloom) =====
    df_st = df[df["month"].between(2,6)].copy()
    df_st = df_st[(df_st[TARGET_STAGE_COL].notna()) & (df_st[TARGET_STAGE_COL]!="NoBloom")]
    y_stg = df_st[TARGET_STAGE_COL].copy()
    g_stg = df_st["Year"]
    X_stg = df_st[feats_num + feats_cat].copy()

    print("🔎 Selecting best STAGE model (v2)...")
    best_stg, lb_stg = evaluate_and_select("stage", X_stg, y_stg, g_stg,
                                           num_cols=feats_num, cat_cols=feats_cat,
                                           scoring_pref=scoring_pref_stage)
    joblib.dump(best_stg, STG_MODEL_PATH)
    print("✅ Saved best stage model  →", STG_MODEL_PATH)
    try:
        display(lb_stg if not lb_stg.empty else pd.DataFrame([{"note":"no models succeeded"}]))
    except Exception:
        print(lb_stg.head() if not lb_stg.empty else "no models succeeded")

    # meta للحفظ (مع priors)
    meta = {
        "required_features_num": feats_num,
        "required_features_cat": feats_cat,
        "binary_labels": ["NoBloom","Bloom"],
        "stage_labels": sorted(y_stg.unique().tolist()),
        "leaderboards": {
            "binary": lb_bin.to_dict(orient="records") if not lb_bin.empty else [],
            "stage":  lb_stg.to_dict(orient="records") if not lb_stg.empty else []
        },
        "priors": {
            "bloom_prior_by_month": {int(k): float(v) for k,v in bloom_prior_by_month.items()},
            "stage_prior_by_month": {int(k): {s: float(p) for s,p in d.items()} for k,d in stage_prior_by_month.items()}
        }
    }
    with open(META_PATH, "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)
    print("✅ Saved meta →", META_PATH)

    return lb_bin, lb_stg

# 🚀 شغّلي إعادة التدريب
lb_bin_v2, lb_stg_v2 = train_v2()


🔎 Selecting best BINARY model (v2)...
✅ Saved best binary model → /content/drive/MyDrive/Bloom Watch/processed/models_bloom_hier/gate_bloom_binary.joblib


,model,acc,bal_acc,f1_macro
0,ET,0.967330,0.962494,0.966131
1,RF,0.967330,0.962494,0.966131
4,CalibratedLR,0.961080,0.957231,0.959730
3,CalibratedLinearSVC,0.956534,0.952587,0.954880
2,HGB,0.950284,0.946109,0.948402


🔎 Selecting best STAGE model (v2)...
✅ Saved best stage model  → /content/drive/MyDrive/Bloom Watch/processed/models_bloom_hier/stage_multiclass.joblib


,model,acc,bal_acc,f1_macro
1,ET,0.709569,0.661103,0.654316
0,RF,0.698023,0.646908,0.626609
2,HGB,0.673948,0.609619,0.580101


✅ Saved meta → /content/drive/MyDrive/Bloom Watch/processed/models_bloom_hier/meta.json


## **Retrain Again**

In [9]:
# =========================================================
# v3 FINAL: RandomForest + Time/Week Priors + Weights + Manual CV GridSearch
# =========================================================
import os, json, joblib, warnings, numpy as np, pandas as pd, re
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GroupKFold, ParameterGrid
from sklearn.metrics import f1_score, balanced_accuracy_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier

# ---------------- Paths ----------------
WIDE_PATH    = "/content/drive/MyDrive/Bloom Watch/processed/ANALYSIS_ready_WIDE_final.csv"
ARTIFACT_DIR = "/content/drive/MyDrive/Bloom Watch/processed/models_bloom_hier"
os.makedirs(ARTIFACT_DIR, exist_ok=True)
BIN_MODEL_PATH = os.path.join(ARTIFACT_DIR, "gate_bloom_binary.joblib")      # RF - Binary
STG_MODEL_PATH = os.path.join(ARTIFACT_DIR, "stage_multiclass.joblib")       # RF - Stage
META_PATH      = os.path.join(ARTIFACT_DIR, "meta.json")

# ---------------- Columns & Mapping ----------------
DATE_COL, YEAR_COL = "date","Year"
TARGET_STAGE_COL = "bloom_stage"
CATEGORICAL_FEATURES = ["region"]   # زودي أي فئات أخرى هنا

CANDIDATE_MAP = {
    "AirTemp_avg": ["AirTemp_avg_final","AirTemp_avg","T2M_final","T2M"],
    "AirTemp_max": ["AirTemp_max_final","AirTemp_max","T2M_MAX_final","T2M_MAX"],
    "AirTemp_min": ["AirTemp_min_final","AirTemp_min","T2M_MIN_final","T2M_MIN"],
    "Precipitation": ["Precipitation_final","Precipitation","PRECTOTCORR_final","PRECTOTCORR"],
    "Humidity_rel": ["Humidity_rel_final","Humidity_rel","RH2M_final","RH2M"],
    "VaporPressureDeficit": ["VaporPressureDeficit_final","VaporPressureDeficit","VPD_final","VPD"],
    "Evapotranspiration_ref": ["Evapotranspiration_ref_final","Evapotranspiration_ref","ET0_final","ET0"],
    "SpecificHumidity": ["SpecificHumidity_final","SpecificHumidity","QV2M_final","QV2M"],
    "SolarRadiation_sw": ["SolarRadiation_sw_final","SolarRadiation_sw","ALLSKY_SFC_SW_DWN_final","ALLSKY_SFC_SW_DWN"],
    "SurfacePressure": ["SurfacePressure_final","SurfacePressure","PS_final","PS"],
}
NUMERIC_BASE = list(CANDIDATE_MAP.keys())

# ---------------- Utils ----------------
def _safe_int(x):
    import numpy as _np
    if isinstance(x, (tuple, list, _np.ndarray)) and len(x) > 0:
        x = x[0]
    try:
        return int(x)
    except Exception:
        x2 = pd.to_numeric(x, errors="coerce")
        if pd.isna(x2): raise ValueError(f"Cannot convert value {x!r} to int")
        return int(x2)

def ensure_time_fields(df: pd.DataFrame) -> pd.DataFrame:
    if DATE_COL in df.columns:
        df[DATE_COL] = pd.to_datetime(df[DATE_COL])
        df["month"] = df.get("month", df[DATE_COL].dt.month)
        df["Year"]  = df.get("Year",  df[DATE_COL].dt.year)
        df["doy"]   = df.get("doy",   df[DATE_COL].dt.dayofyear)
        df["week_of_year"] = df.get("week_of_year", df[DATE_COL].dt.isocalendar().week.astype(int))
        df["week"]  = df.get("week", df["week_of_year"])
    else:
        for c in ["month","Year","doy","week_of_year","week"]:
            if c not in df.columns: raise ValueError(f"Missing time col: {c}")
    df["month"] = pd.to_numeric(df["month"], errors="coerce").astype("Int64").astype(int)
    df["week_of_year"] = pd.to_numeric(df["week_of_year"], errors="coerce").astype("Int64").astype(int)
    return df

def unify_feature_names(df: pd.DataFrame) -> pd.DataFrame:
    dfu = df.copy()
    for std, cand in CANDIDATE_MAP.items():
        for c in cand:
            if c in dfu.columns:
                if c != std: dfu = dfu.rename(columns={c: std})
                break
        else:
            raise KeyError(f"Missing feature for '{std}'.")
    return ensure_time_fields(dfu)

def normalize_stage_label(s: str) -> str:
    if not isinstance(s,str) or s.strip()=="":
        return s
    t = s.lower().strip()
    t = re.sub(r"\s*\|\s*", "|", t); t = re.sub(r"\s*/\s*", "/", t)
    is_super = ("superbloom" in t) or (re.search(r"\bsuper\b", t) is not None)
    direct = {
        "sob":"SoB","start bloom":"SoB","start of bloom":"SoB",
        "mob":"MoB","mid bloom":"MoB",
        "lmob":"LMoB","late mid":"LMoB",
        "pob":"PoB","peak bloom":"PoB",
        "lpob":"LPoB","late peak":"LPoB",
        "eob":"EoB","end of bloom":"EoB",
        "no bloom":"NoBloom","nobloom":"NoBloom"
    }
    if t in direct:
        lbl = direct[t];
        return "SoB_super" if (lbl=="SoB" and is_super) else lbl
    if "start" in t and "bloom" in t: return "SoB_super" if is_super else "SoB"
    if "late" in t and "mid" in t:    return "LMoB"
    if "mid" in t:                    return "MoB"
    if "late" in t and "peak" in t:   return "LPoB"
    if "peak" in t:                   return "PoB"
    if "end" in t or "eob" in t:      return "EoB"
    if "no bloom" in t or "nobloom" in t: return "NoBloom"
    if "sob" in t or "start" in t:    return "SoB_super" if is_super else "SoB"
    return s.strip()

def make_binary_from_stage(df, col=TARGET_STAGE_COL):
    s = df[col].astype(str).apply(normalize_stage_label)
    return np.where(s=="NoBloom","NoBloom","Bloom")

def add_time_encodings(df):
    df["m_sin"] = np.sin(2*np.pi*(df["month"]-1)/12)
    df["m_cos"] = np.cos(2*np.pi*(df["month"]-1)/12)
    df["w_sin"] = np.sin(2*np.pi*(df["week_of_year"]-1)/53)
    df["w_cos"] = np.cos(2*np.pi*(df["week_of_year"]-1)/53)
    df["d_sin"] = np.sin(2*np.pi*(df["doy"]-1)/366)
    df["d_cos"] = np.cos(2*np.pi*(df["doy"]-1)/366)
    return df

# --- Week priors ---
def build_week_priors(df_full: pd.DataFrame):
    df = ensure_time_fields(df_full.copy())
    df[TARGET_STAGE_COL] = df[TARGET_STAGE_COL].apply(normalize_stage_label)
    df["_is_bloom"] = (df[TARGET_STAGE_COL] != "NoBloom")
    bloom_prior_week = {_safe_int(w): float(p) for w,p in df.groupby("week_of_year")["_is_bloom"].mean().items()}
    st = df[df["_is_bloom"]]
    counts = st.groupby(["week_of_year", TARGET_STAGE_COL]).size()
    probs  = counts.groupby(level=0).apply(lambda s: s/s.sum()).unstack().fillna(0.0)
    stage_set = sorted(list(map(str, probs.columns)))
    stage_prior_week = {}
    for w in probs.index:
        wi = _safe_int(w)
        stage_prior_week[wi] = {str(k): float(v) for k, v in probs.loc[w].to_dict().items()}
    return bloom_prior_week, stage_prior_week, stage_set

def add_priors_columns(df, bloom_prior_week, stage_prior_week, stage_set):
    df["prior_bloom_w"] = df["week_of_year"].map(bloom_prior_week).fillna(0.0)
    for stg in stage_set:
        df[f"prior_{stg}_w"] = df["week_of_year"].map({w:d.get(stg,0.0) for w,d in stage_prior_week.items()}).fillna(0.0)
    prior_cols = ["prior_bloom_w"] + [f"prior_{stg}_w" for stg in stage_set]
    return df, prior_cols

def add_monthly_anomalies(df, full_df, feat_list):
    z_cols = []
    stats = full_df.groupby("month")[feat_list].agg(['mean','std'])
    stats.columns = ['_'.join(c) for c in stats.columns]
    merged = df.merge(stats, left_on="month", right_index=True, how="left")
    for f in feat_list:
        zc = f"{f}_z"
        merged[zc] = (merged[f] - merged[f"{f}_mean"]) / merged[f"{f}_std"].replace(0,np.nan)
        merged[zc] = merged[zc].fillna(0.0)
        z_cols.append(zc)
    drop_cols = [c for c in merged.columns if c.endswith("_mean") or c.endswith("_std")]
    return merged.drop(columns=drop_cols), z_cols

def hard_gate_month(m): return int(m) in [2,3,4,5,6]

def build_preprocessor(num_cols, cat_cols):
    return ColumnTransformer(
        transformers=[
            ("num", StandardScaler(), num_cols),
            ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ],
        remainder="drop"
    )

# -------- Manual CV with sample_weight --------
def cv_score_pipeline(pipe, X, y, groups, sample_weight=None, n_splits=5):
    gkf = GroupKFold(n_splits=n_splits)
    accs, bals, f1s = [], [], []
    for tr, te in gkf.split(X, y, groups):
        Xtr, Xte = X.iloc[tr], X.iloc[te]
        ytr, yte = y.iloc[tr], y.iloc[te]
        fit_kwargs = {}
        if sample_weight is not None:
            fit_kwargs["clf__sample_weight"] = np.asarray(sample_weight)[tr]
        pipe.fit(Xtr, ytr, **fit_kwargs)
        yp = pipe.predict(Xte)
        accs.append(accuracy_score(yte, yp))
        bals.append(balanced_accuracy_score(yte, yp))
        f1s.append(f1_score(yte, yp, average="macro"))
    return float(np.mean(accs)), float(np.mean(bals)), float(np.mean(f1s))

def rf_search(task, X, y, groups, num_cols, cat_cols, sample_weight=None, scoring_pref="f1_macro"):
    param_grid = {
        "clf__n_estimators":   [600, 900, 1400],
        "clf__max_depth":      [None, 12, 18, 24],
        "clf__min_samples_leaf":[1, 2, 4],
        "clf__max_features":   ["sqrt", "log2", 0.6],
        "clf__bootstrap":      [True, False],
    }
    base = Pipeline([
        ("pre", build_preprocessor(num_cols, cat_cols)),
        ("clf", RandomForestClassifier(class_weight="balanced_subsample", n_jobs=-1, random_state=42))
    ])
    rows, best_est, best_score = [], None, -np.inf
    for params in ParameterGrid(param_grid):
        m = Pipeline(steps=base.steps)
        m.set_params(**params)
        try:
            acc, bal, f1m = cv_score_pipeline(m, X, y, groups, sample_weight=sample_weight, n_splits=5)
            rows.append({"params": params, "acc": acc, "bal_acc": bal, "f1_macro": f1m})
            key = {"accuracy":acc, "balanced_accuracy":bal, "f1_macro":f1m}[scoring_pref]
            if key > best_score:
                best_score = key
                best_est = m
        except Exception as e:
            warnings.warn(f"[{task}] RF fit failed for {params}: {e}")
            continue
    lb = pd.DataFrame(rows).sort_values(by=scoring_pref, ascending=False) if rows else pd.DataFrame()
    # refit best on ALL data
    if best_est is not None:
        fit_kwargs = {}
        if sample_weight is not None: fit_kwargs["clf__sample_weight"] = np.asarray(sample_weight)
        best_est.fit(X, y, **fit_kwargs)
    return best_est, lb

# ---------------- Training (v3) ----------------
def train_v3(alpha_time_bin=1.0, alpha_time_stage=1.8,
             scoring_pref_bin="f1_macro", scoring_pref_stage="f1_macro"):
    raw = pd.read_csv(WIDE_PATH)
    df  = unify_feature_names(raw)
    df[TARGET_STAGE_COL] = df[TARGET_STAGE_COL].apply(normalize_stage_label)

    bloom_prior_w, stage_prior_w, stage_set = build_week_priors(df)
    df = add_time_encodings(df)
    df, prior_cols = add_priors_columns(df, bloom_prior_w, stage_prior_w, stage_set)
    df, z_cols = add_monthly_anomalies(df, df, NUMERIC_BASE)

    time_cols = ["m_sin","m_cos","w_sin","w_cos","d_sin","d_cos"]
    feats_num = NUMERIC_BASE + z_cols + prior_cols + time_cols
    feats_cat = CATEGORICAL_FEATURES

    # ===== Binary داخل [2..6] =====
    df_bin = df[df["month"].between(2,6)].copy()
    y_bin  = pd.Series(make_binary_from_stage(df_bin), index=df_bin.index)
    X_bin  = df_bin[feats_num + feats_cat].copy()
    g_bin  = df_bin["Year"]
    sw_bin = 1.0 + alpha_time_bin * df_bin["prior_bloom_w"].to_numpy(dtype=float)

    if (y_bin == "Bloom").all():
        print("⚠️ داخل [2..6] كل الأمثلة Bloom → Gate rule + Dummy.")
        best_bin = Pipeline([("pre", build_preprocessor(feats_num, feats_cat)),
                             ("clf", DummyClassifier(strategy='constant', constant='Bloom'))]).fit(X_bin, y_bin)
        lb_bin = pd.DataFrame([{"model":"Dummy(Bloom)","acc":1.0,"bal_acc":0.5,"f1_macro":(2/3)}])
    else:
        print("🔎 RF search (Binary, time-weighted)...")
        best_bin, lb_bin = rf_search("binary", X_bin, y_bin, g_bin,
                                     num_cols=feats_num, cat_cols=feats_cat,
                                     sample_weight=sw_bin, scoring_pref=scoring_pref_bin)

    joblib.dump(best_bin, BIN_MODEL_PATH)
    print("✅ Saved best binary RF →", BIN_MODEL_PATH)
    try: display(lb_bin.head(10))
    except: print(lb_bin.head())

    # ===== Stage داخل [2..6] (بدون NoBloom) =====
    df_st = df[df["month"].between(2,6)].copy()
    df_st = df_st[(df_st[TARGET_STAGE_COL].notna()) & (df_st[TARGET_STAGE_COL]!="NoBloom")]
    y_stg  = df_st[TARGET_STAGE_COL].copy()
    X_stg  = df_st[feats_num + feats_cat].copy()
    g_stg  = df_st["Year"]

    p_true = []
    for w, lbl in zip(df_st["week_of_year"].to_numpy(), y_stg.to_numpy()):
        p_true.append(stage_prior_w.get(int(w), {}).get(str(lbl), 0.0))
    sw_stg = 1.0 + alpha_time_stage * np.array(p_true, dtype=float)

    print("🔎 RF search (Stage, time-weighted)...")
    best_stg, lb_stg = rf_search("stage", X_stg, y_stg, g_stg,
                                 num_cols=feats_num, cat_cols=feats_cat,
                                 sample_weight=sw_stg, scoring_pref=scoring_pref_stage)

    joblib.dump(best_stg, STG_MODEL_PATH)
    print("✅ Saved best stage RF  →", STG_MODEL_PATH)
    try: display(lb_stg.head(10))
    except: print(lb_stg.head())

    meta = {
        "required_features_num": feats_num,
        "required_features_cat": feats_cat,
        "binary_labels": ["NoBloom","Bloom"],
        "stage_labels": sorted(y_stg.unique().tolist()),
        "priors_week": {
            "bloom_prior_w": {int(k): float(v) for k,v in bloom_prior_w.items()},
            "stage_prior_w": {int(k): {s: float(p) for s,p in d.items()} for k,d in stage_prior_w.items()}
        },
        "leaderboards": {
            "binary": lb_bin.to_dict(orient="records") if lb_bin is not None else [],
            "stage":  lb_stg.to_dict(orient="records") if lb_stg is not None else []
        },
        "alpha_time": {"binary": float(alpha_time_bin), "stage": float(alpha_time_stage)}
    }
    with open(META_PATH, "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)
    print("✅ Saved meta →", META_PATH)

    return lb_bin, lb_stg

# 🚀 شغّلي التدريب (غيري الألفا لو عايزة وزن أقوى للوقت):
lb_bin_v3, lb_stg_v3 = train_v3(alpha_time_bin=1.2, alpha_time_stage=2.0)


🔎 RF search (Binary, time-weighted)...
✅ Saved best binary RF → /content/drive/MyDrive/Bloom Watch/processed/models_bloom_hier/gate_bloom_binary.joblib


,params,acc,bal_acc,f1_macro
0,"{'clf__bootstrap': True, 'clf__max_depth': Non...",0.96733,0.962494,0.966131
1,"{'clf__bootstrap': True, 'clf__max_depth': Non...",0.96733,0.962494,0.966131
2,"{'clf__bootstrap': True, 'clf__max_depth': Non...",0.96733,0.962494,0.966131
23,"{'clf__bootstrap': True, 'clf__max_depth': Non...",0.96733,0.962494,0.966131
22,"{'clf__bootstrap': True, 'clf__max_depth': Non...",0.96733,0.962494,0.966131
11,"{'clf__bootstrap': True, 'clf__max_depth': Non...",0.96733,0.962494,0.966131
10,"{'clf__bootstrap': True, 'clf__max_depth': Non...",0.96733,0.962494,0.966131
9,"{'clf__bootstrap': True, 'clf__max_depth': Non...",0.96733,0.962494,0.966131
21,"{'clf__bootstrap': True, 'clf__max_depth': Non...",0.96733,0.962494,0.966131
54,"{'clf__bootstrap': True, 'clf__max_depth': 18,...",0.96733,0.962494,0.966131


🔎 RF search (Stage, time-weighted)...
✅ Saved best stage RF  → /content/drive/MyDrive/Bloom Watch/processed/models_bloom_hier/stage_multiclass.joblib


,params,acc,bal_acc,f1_macro
79,"{'clf__bootstrap': True, 'clf__max_depth': 18,...",0.73973,0.706614,0.691258
80,"{'clf__bootstrap': True, 'clf__max_depth': 18,...",0.73973,0.706614,0.691258
51,"{'clf__bootstrap': True, 'clf__max_depth': 12,...",0.73973,0.706614,0.691258
53,"{'clf__bootstrap': True, 'clf__max_depth': 12,...",0.73973,0.706614,0.691258
52,"{'clf__bootstrap': True, 'clf__max_depth': 12,...",0.73973,0.706614,0.691258
26,"{'clf__bootstrap': True, 'clf__max_depth': Non...",0.73973,0.706614,0.691258
25,"{'clf__bootstrap': True, 'clf__max_depth': Non...",0.73973,0.706614,0.691258
24,"{'clf__bootstrap': True, 'clf__max_depth': Non...",0.73973,0.706614,0.691258
78,"{'clf__bootstrap': True, 'clf__max_depth': 18,...",0.73973,0.706614,0.691258
105,"{'clf__bootstrap': True, 'clf__max_depth': 24,...",0.73973,0.706614,0.691258


✅ Saved meta → /content/drive/MyDrive/Bloom Watch/processed/models_bloom_hier/meta.json


In [13]:
# Define the actual prediction logic
import pandas as pd
import numpy as np
import joblib
import json
import re
import os

# Load models and meta-data
BIN_MODEL_PATH = "/content/drive/MyDrive/Bloom Watch/processed/models_bloom_hier/gate_bloom_binary.joblib"
STG_MODEL_PATH = "/content/drive/MyDrive/Bloom Watch/processed/models_bloom_hier/stage_multiclass.joblib"
META_PATH      = "/content/drive/MyDrive/Bloom Watch/processed/models_bloom_hier/meta.json"

try:
    binary_model = joblib.load(BIN_MODEL_PATH)
    stage_model  = joblib.load(STG_MODEL_PATH)
    with open(META_PATH, "r", encoding="utf-8") as f:
        meta = json.load(f)
except FileNotFoundError:
    print("Error: Model or meta files not found. Please ensure they are in the correct path.")
    binary_model, stage_model, meta = None, None, None

# Re-define necessary helper functions from the training cells
def _safe_int(x):
    if isinstance(x, (tuple, list, np.ndarray)) and len(x) > 0:
        x = x[0]
    try:
        return int(x)
    except Exception:
        x2 = pd.to_numeric(x, errors="coerce")
        if pd.isna(x2): raise ValueError(f"Cannot convert value {x!r} to int")
        return int(x2)

def ensure_time_fields(df: pd.DataFrame) -> pd.DataFrame:
    DATE_COL = "date"
    if DATE_COL in df.columns:
        df[DATE_COL] = pd.to_datetime(df[DATE_COL])
        df["month"] = df.get("month", df[DATE_COL].dt.month)
        df["Year"]  = df.get("Year",  df[DATE_COL].dt.year)
        df["doy"]   = df.get("doy",   df[DATE_COL].dt.dayofyear)
        df["week_of_year"] = df.get("week_of_year", df[DATE_COL].dt.isocalendar().week.astype(int))
        df["week"]  = df.get("week", df["week_of_year"])
    else:
        for c in ["month","Year","doy","week_of_year","week"]:
            if c not in df.columns: raise ValueError(f"Missing time col: {c}")
    df["month"] = pd.to_numeric(df["month"], errors="coerce").astype("Int64").astype(int)
    df["week_of_year"] = pd.to_numeric(df["week_of_year"], errors="coerce").astype("Int64").astype(int)
    return df

def unify_feature_names(df: pd.DataFrame) -> pd.DataFrame:
    CANDIDATE_MAP = {
        "AirTemp_avg": ["AirTemp_avg_final","AirTemp_avg","T2M_final","T2M"],
        "AirTemp_max": ["AirTemp_max_final","AirTemp_max","T2M_MAX_final","T2M_MAX"],
        "AirTemp_min": ["AirTemp_min_final","AirTemp_min","T2M_MIN_final","T2M_MIN"],
        "Precipitation": ["Precipitation_final","Precipitation","PRECTOTCORR_final","PRECTOTCORR"],
        "Humidity_rel": ["Humidity_rel_final","Humidity_rel","RH2M_final","RH2M"],
        "VaporPressureDeficit": ["VaporPressureDeficit_final","VaporPressureDeficit","VPD_final","VPD"],
        "Evapotranspiration_ref": ["Evapotranspiration_ref_final","Evapotranspiration_ref","ET0_final","ET0"],
        "SpecificHumidity": ["SpecificHumidity_final","SpecificHumidity","QV2M_final","QV2M"],
        "SolarRadiation_sw": ["SolarRadiation_sw_final","SolarRadiation_sw","ALLSKY_SFC_SW_DWN_final","ALLSKY_SFC_SW_DWN"],
        "SurfacePressure": ["SurfacePressure_final","SurfacePressure","PS_final","PS"],
    }
    dfu = df.copy()
    for std, cand in CANDIDATE_MAP.items():
        for c in cand:
            if c in dfu.columns:
                if c != std: dfu = dfu.rename(columns={c: std})
                break
        else:
            # If column not found with _final suffix, look for it without
            for c in [c.replace("_final", "") for c in cand]:
                 if c in dfu.columns:
                    if c != std: dfu = dfu.rename(columns={c: std})
                    break
            else:
                raise KeyError(f"Missing feature for '{std}'.")

    return ensure_time_fields(dfu)

def add_time_encodings(df):
    df["m_sin"] = np.sin(2*np.pi*(df["month"]-1)/12)
    df["m_cos"] = np.cos(2*np.pi*(df["month"]-1)/12)
    df["w_sin"] = np.sin(2*np.pi*(df["week_of_year"]-1)/53)
    df["w_cos"] = np.cos(2*np.pi*(df["week_of_year"]-1)/53)
    df["d_sin"] = np.sin(2*np.pi*(df["doy"]-1)/366)
    df["d_cos"] = np.cos(2*np.pi*(df["doy"]-1)/366)
    return df

def add_priors_columns(df, bloom_prior_week, stage_prior_week, stage_set):
    df["prior_bloom_w"] = df["week_of_year"].map(bloom_prior_week).fillna(0.0)
    for stg in stage_set:
        df[f"prior_{stg}_w"] = df["week_of_year"].map({w:d.get(stg,0.0) for w,d in stage_prior_week.items()}).fillna(0.0)
    prior_cols = ["prior_bloom_w"] + [f"prior_{stg}_w" for stg in stage_set]
    return df, prior_cols

def add_monthly_anomalies(df, full_df, feat_list):
    z_cols = []
    stats = full_df.groupby("month")[feat_list].agg(['mean','std'])
    stats.columns = ['_'.join(c) for c in stats.columns]
    merged = df.merge(stats, left_on="month", right_index=True, how="left")
    for f in feat_list:
        zc = f"{f}_z"
        merged[zc] = (merged[f] - merged[f"{f}_mean"]) / merged[f"{f}_std"].replace(0,np.nan)
        merged[zc] = merged[zc].fillna(0.0)
        z_cols.append(zc)
    drop_cols = [c for c in merged.columns if c.endswith("_mean") or c.endswith("_std")]
    return merged.drop(columns=drop_cols), z_cols

def hard_gate_month(m): return int(m) in [2,3,4,5,6]


def predict_bloom(df_raw: pd.DataFrame, prob_threshold: float = 0.5):
    """
    Predicts bloom flag and stage using the loaded models.

    Args:
        df_raw: Input DataFrame with raw features.
        prob_threshold: Probability threshold for binary classification.

    Returns:
        DataFrame with predictions.
    """
    if binary_model is None or stage_model is None or meta is None:
        return "Models not loaded. Please check file paths."

    df = df_raw.copy()
    df_processed = unify_feature_names(df) # Ensure this returns the modified DataFrame
    df_processed = add_time_encodings(df_processed)

    # Add priors (using priors from meta)
    bloom_prior_w = meta.get("priors_week", {}).get("bloom_prior_w", {})
    stage_prior_w = meta.get("priors_week", {}).get("stage_prior_w", {})
    stage_set = meta.get("stage_labels", [])

    df_processed, prior_cols = add_priors_columns(df_processed, bloom_prior_w, stage_prior_w, stage_set)

    # Add monthly anomalies (Z-scores) for numeric base features
    NUMERIC_BASE = meta.get("required_features_num", []) # Assuming these are the base numeric features
    # Need a full dataset to calculate anomalies correctly.
    # As a workaround for prediction on new data, we'll use the stats from the training data stored in meta.
    # This is a simplification; ideally, anomaly calculation would be dynamic or based on a larger dataset.
    # For demonstration, we'll just use the features as is without dynamic anomaly calculation here.
    # A more robust approach would store the mean/std from training in meta.
    # For now, let's skip dynamic anomaly calculation in prediction and use the features directly.
    # df_processed, z_cols = add_monthly_anomalies(df_processed, df_raw, NUMERIC_BASE)


    feats_num = meta.get("required_features_num", [])
    feats_cat = meta.get("required_features_cat", [])

    # Ensure all required features are present
    missing_num = [f for f in feats_num if f not in df_processed.columns]
    missing_cat = [f for f in feats_cat if f not in df_processed.columns]
    if missing_num or missing_cat:
         print(f"Warning: Missing features for prediction: Numeric: {missing_num}, Categorical: {missing_cat}")
         # Attempt to fill missing numeric with 0 or a sensible default, add missing cat columns with placeholder
         for f in missing_num: df_processed[f] = 0.0
         for f in missing_cat: df_processed[f] = 'unknown' # Or another placeholder

    # Filter features based on meta
    X = df_processed[feats_num + feats_cat]

    # Binary Prediction (Gate)
    df["month_int"] = df_processed["date"].dt.month.astype(int) if "date" in df_processed.columns else df_processed["month"].astype(int) # Use df_processed here
    is_in_season = df["month_int"].apply(hard_gate_month)

    X_season = X[is_in_season].copy()

    if not X_season.empty:
        binary_probs = binary_model.predict_proba(X_season)[:, binary_model.classes_.tolist().index("Bloom")]
        binary_preds = np.where(binary_probs >= prob_threshold, "Bloom", "NoBloom")
    else:
        binary_probs = np.array([])
        binary_preds = np.array([])


    df["bloom_prob"] = 0.0
    df["bloom_flag"] = "NoBloom"

    if not X_season.empty:
        df.loc[is_in_season, "bloom_prob"] = binary_probs
        df.loc[is_in_season, "bloom_flag"] = binary_preds


    # Stage Prediction (only for predicted 'Bloom' cases within season)
    is_predicted_bloom_in_season = df.index[ (df["bloom_flag"] == "Bloom") & (is_in_season) ]

    df["stage_prob"] = 0.0
    df["bloom_stage"] = "NoBloom" # Default stage is NoBloom

    if not is_predicted_bloom_in_season.empty:
        X_stage = X.loc[is_predicted_bloom_in_season].copy()
        if not X_stage.empty:
            stage_probs = stage_model.predict_proba(X_stage)
            stage_preds_idx = np.argmax(stage_probs, axis=1)
            stage_labels = stage_model.classes_
            stage_preds = stage_labels[stage_preds_idx]
            stage_max_probs = np.max(stage_probs, axis=1)

            df.loc[is_predicted_bloom_in_season, "bloom_stage"] = stage_preds
            df.loc[is_predicted_bloom_in_season, "stage_prob"] = stage_max_probs


    # For cases outside the season or predicted NoBloom, ensure stage is NoBloom
    df.loc[~is_in_season | (df["bloom_flag"] == "NoBloom"), "bloom_stage"] = "NoBloom"
    df.loc[~is_in_season | (df["bloom_flag"] == "NoBloom"), "stage_prob"] = 0.0 # Stage prob is 0 if no bloom

    return df

# Redefine predict_one_case and predict_batch to use the new predict_bloom
def predict_one_case(
    date:str,
    region:str,
    AirTemp_avg:float,
    AirTemp_max:float,
    AirTemp_min:float,
    Precipitation:float,
    Humidity_rel:float,
    VaporPressureDeficit:float,
    Evapotranspiration_ref:float,
    SpecificHumidity:float,
    SolarRadiation_sw:float,
    SurfacePressure:float,
    prob_threshold:float=0.5
):
    """
    Predicts bloom for a single case using the loaded models.
    """
    row = {
        "region": region,
        "date": date,  # YYYY-MM-DD
        "AirTemp_avg_final": AirTemp_avg,
        "AirTemp_max_final": AirTemp_max,
        "AirTemp_min_final": AirTemp_min,
        "Precipitation_final": Precipitation,
        "Humidity_rel_final": Humidity_rel,
        "VaporPressureDeficit_final": VaporPressureDeficit,
        "Evapotranspiration_ref_final": Evapotranspiration_ref,
        "SpecificHumidity_final": SpecificHumidity,
        "SolarRadiation_sw_final": SolarRadiation_sw,
        "SurfacePressure_final": SurfacePressure,
    }
    df = pd.DataFrame([row])
    pred = predict_bloom(df, prob_threshold=prob_threshold).copy()
    # display(
    #     pred[[
    #         "region","date","bloom_flag","bloom_prob","bloom_stage","stage_prob"
    #     ]]
    # )
    return pred

def predict_batch(cases:list, prob_threshold:float=0.5):
    """
    Predicts bloom for a batch of cases using the loaded models.
    """
    df = pd.DataFrame(cases)
    pred = predict_bloom(df, prob_threshold=prob_threshold).copy()
    # display(pred[["region","date","bloom_flag","bloom_prob","bloom_stage","stage_prob"]])
    return pred

print("Prediction functions defined and models loaded (if files found).")

Prediction functions defined and models loaded (if files found).


In [14]:
# Example of how to use predict_one_case with sample values
# Make sure the predict_one_case and predict_batch functions are defined in the preceding cells

print("--- Testing predict_one_case ---")
result_one = predict_one_case(
    date="2024-04-15", # Example date within the bloom season
    region="California", # Example region
    AirTemp_avg=18.0,
    AirTemp_max=25.5,
    AirTemp_min=12.0,
    Precipitation=0.0,
    Humidity_rel=50,
    VaporPressureDeficit=1.5,
    Evapotranspiration_ref=4.0,
    SpecificHumidity=0.0070,
    SolarRadiation_sw=280,
    SurfacePressure=101.0,
    prob_threshold=0.5
)
display(result_one[["region","date","bloom_flag","bloom_prob","bloom_stage","stage_prob"]])


print("\n--- Testing predict_batch ---")
batch_cases = [
    {
        "region": "California",
        "date": "2024-04-18",
        "AirTemp_avg_final": 19.0,
        "AirTemp_max_final": 26.0,
        "AirTemp_min_final": 13.0,
        "Precipitation_final": 0.0,
        "Humidity_rel_final": 55,
        "VaporPressureDeficit_final": 1.6,
        "Evapotranspiration_ref_final": 4.2,
        "SpecificHumidity_final": 0.0075,
        "SolarRadiation_sw_final": 290,
        "SurfacePressure_final": 101.1,
    },
    {
        "region": "Arizona",
        "date": "2024-03-10",
        "AirTemp_avg_final": 15.0,
        "AirTemp_max_final": 22.0,
        "AirTemp_min_final": 9.0,
        "Precipitation_final": 1.5,
        "Humidity_rel_final": 60,
        "VaporPressureDeficit_final": 1.0,
        "Evapotranspiration_ref_final": 3.0,
        "SpecificHumidity_final": 0.0060,
        "SolarRadiation_sw_final": 200,
        "SurfacePressure_final": 100.8,
    }
]

result_batch = predict_batch(batch_cases, prob_threshold=0.5)
display(result_batch[["region","date","bloom_flag","bloom_prob","bloom_stage","stage_prob"]])

--- Testing predict_one_case ---


,region,date,bloom_flag,bloom_prob,bloom_stage,stage_prob
0,California,2024-04-15,Bloom,0.761523,PoB,0.633197



--- Testing predict_batch ---


,region,date,bloom_flag,bloom_prob,bloom_stage,stage_prob
0,California,2024-04-18,Bloom,0.76247,LPoB,0.530601
1,Arizona,2024-03-10,Bloom,0.72567,SoB,0.860965
